In [14]:
import yaml
import textwrap

class Registry(list):
    """Registry class"""
    def get_cls(self, tag):
        """Get class by tag"""
        for cls in self:
            if tag == cls.tag:
                return cls
        raise KeyError(f"No object found with tag: {tag!r}")

        
class TypedAttribute:
    """Typed attribute"""        
    def __init__(self, default=None , info=None, json_decode=None, type_=None):
        self.default = default            
        self.info = info
        self.json_decode = json_decode
        
        if type_ is None:
            self._type_ = type(default)
        else:
            raise ValueError("Default value required")
    
    @property
    def type_(self):
        return type(self.default)
    
    # not sure this is safe...
    @property
    def __doc__(self):
        return self.info
        
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        return instance.__dict__.get("_value_" + self.name, self.default)

    def __set__(self, instance, value):
        if not type(value) == type(self.default):
            raise ValueError(f"Wrong type {type(value)}, expected {type(self.default)}")
            
        instance.__dict__["_value_" + self.name] = value
    
    def to_json(self):
        """To json"""
        return self.json_decode(self)

    
class Configurable:
    """Configurable with typed attributes"""  
    def __init__(self, **kwargs):
        for name, value in kwargs.items():
            setattr(self, name, value)
    
    @classmethod
    def typed_attributes(cls):
        attributes = {}
        
        for name, value in cls.__dict__.items():
            if isinstance(value, TypedAttribute):
                attributes[name] = value
        
        return attributes
    
    def __setattr__(self, name, value):
        attribute = getattr(self.__class__, name, None)
        
        if attribute is None:
            raise AttributeError(f"Not a valid attribute: '{name}'")
            
        attribute.__set__(self, value)

    def __str__(self):
        info = self.__class__.__name__
        info += "\n" + len(info) * "-" + "\n\n"
        
        for name, attribute in self.typed_attributes().items():
            value = getattr(self, name)
            if isinstance(value, Configurable):
                info += f"\t{name}:\n\n" + textwrap.indent(str(value), prefix="\t\t") + "\n"
            else:
                info += f"\t{name}: {getattr(self, name)} ({attribute.info}) \n"
            
        return info.expandtabs(tabsize=2)
    
    def to_json(self):
        """To json"""
        data = {}
        
        for name, value in self.typed_attributes().items():
            value_attr = getattr(self, name)
            
            if isinstance(value_attr, Configurable):
                data[name] = value_attr.to_json()  
            else:
                data[name] = value_attr
            
        return {self.tag: data}
    
    def to_yaml(self, sort_keys=True):
        """To yaml"""
        data = self.to_json()
        return yaml.dump(data, default_flow_style=False, sort_keys=sort_keys)
        
    @classmethod
    def from_json(cls, data):
        """From json"""
        if isinstance(data, dict) and len(data) == 1:
            cls_name, data = data.popitem()
            cls = CONFIGURABLE.get_cls(cls_name)
        
        kwargs = {}
                
        for name, value in data.items():
            try:
                value = cls.from_json(value)
            except (KeyError, AttributeError):
                pass
            
            kwargs[name] = value
        
        return cls(**kwargs) 
        
        
class MyData(Configurable):
    """My data"""
    tag = "my-data"
    value_int = TypedAttribute(default=42, info="This the answer to everything")
    value_str = TypedAttribute(default="abc", info="Just the start of the alphabet")
    
    
class MyOtherData(Configurable):
    """My other data"""
    tag = "my-other-data"
    value_cls = TypedAttribute(default=MyData(), info="MyData as attribute")
    value_int = TypedAttribute(default=123)

    
class MyComplexData(Configurable):
    tag = "my-complex-data"
    value_1 = TypedAttribute(default=MyOtherData())
    value_2 = TypedAttribute(default=MyData())

CONFIGURABLE = Registry([MyData, MyOtherData, MyComplexData])

In [15]:
#MyData.value_int?

In [16]:
data = MyData()
data.value_int = 3455

In [17]:
print(data)

MyData
------

  value_int: 3455 (This the answer to everything) 
  value_str: abc (Just the start of the alphabet) 



In [18]:
#data.value_int = "asdd"

In [19]:
data_other = MyOtherData()
data_other.value_cls = MyData()

In [20]:
print(data_other)

MyOtherData
-----------

  value_cls:

    MyData
    ------

      value_int: 42 (This the answer to everything) 
      value_str: abc (Just the start of the alphabet) 

  value_int: 123 (None) 



In [21]:
data_other.value_cls.value_int = 45

In [22]:
json_data = data_other.to_json()

In [23]:
c = Configurable.from_json(json_data)
print(c)

MyOtherData
-----------

  value_cls:

    MyData
    ------

      value_int: 45 (This the answer to everything) 
      value_str: abc (Just the start of the alphabet) 

  value_int: 123 (None) 



In [26]:
data_complex = MyComplexData()
c = Configurable.from_json(data_complex.to_json())
print(c)

MyComplexData
-------------

  value_1:

    MyOtherData
    -----------

      value_cls:

        MyData
        ------

          value_int: 42 (This the answer to everything) 
          value_str: abc (Just the start of the alphabet) 

      value_int: 123 (None) 

  value_2:

    MyData
    ------

      value_int: 42 (This the answer to everything) 
      value_str: abc (Just the start of the alphabet) 




In [ ]:
class Te